# df

In [1]:
CSZ=500

In [2]:
import geopandas as gd
from shapely.geometry import box

In [3]:
gdfin = gd.GeoDataFrame.from_file('AOI/conus_ard_grid.shp')

In [4]:
gdfin.crs

<Projected CRS: PROJCS["Albers_Conic_Equal_Area",GEOGCS["WGS 84",D ...>
Name: Albers_Conic_Equal_Area
Axis Info [cartesian]:
- [east]: Easting (metre)
- [north]: Northing (metre)
Area of Use:
- undefined
Coordinate Operation:
- name: unnamed
- method: Albers Equal Area
Datum: World Geodetic System 1984
- Ellipsoid: WGS 84
- Prime Meridian: Greenwich

In [5]:
H=16
V=4

In [6]:
hvgdf = gdfin.loc[(gdfin['h'] == 16) & (gdfin['v'] == 4)]

In [7]:
geo = hvgdf['geometry'].values[0]

In [8]:
type(geo)

shapely.geometry.polygon.Polygon

In [9]:
geo.bounds

(-165584.99992983043,
 2564804.9997692876,
 -15584.999929830432,
 2714804.9997692876)

In [10]:
def make_superChip_df(h, v, sY, sX, tile_miny, tile_minx, chunksz_pixels, crs):
    chunk_sz_meters = chunksz_pixels * 30
    yoff = sY * chunk_sz_meters
    xoff = sX * chunk_sz_meters

    print(yoff, xoff)
    
    new_minx = int(tile_minx+xoff)
    new_miny = int(tile_miny+yoff)

    new_maxx = int(new_minx + chunk_sz_meters)
    new_maxy = int(new_miny + chunk_sz_meters)

    my_geometry = box(new_minx, new_miny, new_maxx, new_maxy)

    my_df_list=[]
    rec = {
        'H': h,
        'V': v,
        'sY': sY,
        'sX': sX,
        'geometry': my_geometry
    }
    my_df_list.append(rec)
    
    gdf1 = gd.GeoDataFrame(my_df_list)
    gdf = gdf1.set_crs(crs)
    return gdf

In [11]:
tile_minx=geo.bounds[0]
tile_miny=geo.bounds[1]

In [12]:
my_dfs=[]
chunksz_pixels=CSZ
crs=hvgdf.crs
for x in range(int(5000/CSZ)):
    for y in range(int(5000/CSZ)):
        # print(y,x)
        gdfhvyx = make_superChip_df(H, V, y, x, tile_miny, tile_minx, chunksz_pixels, crs)
        my_dfs.append(gdfhvyx)

0 0
15000 0
30000 0
45000 0
60000 0
75000 0
90000 0
105000 0
120000 0
135000 0
0 15000
15000 15000
30000 15000
45000 15000
60000 15000
75000 15000
90000 15000
105000 15000
120000 15000
135000 15000
0 30000
15000 30000
30000 30000
45000 30000
60000 30000
75000 30000
90000 30000
105000 30000
120000 30000
135000 30000
0 45000
15000 45000
30000 45000
45000 45000
60000 45000
75000 45000
90000 45000
105000 45000
120000 45000
135000 45000
0 60000
15000 60000
30000 60000
45000 60000
60000 60000
75000 60000
90000 60000
105000 60000
120000 60000
135000 60000
0 75000
15000 75000
30000 75000
45000 75000
60000 75000
75000 75000
90000 75000
105000 75000
120000 75000
135000 75000
0 90000
15000 90000
30000 90000
45000 90000
60000 90000
75000 90000
90000 90000
105000 90000
120000 90000
135000 90000
0 105000
15000 105000
30000 105000
45000 105000
60000 105000
75000 105000
90000 105000
105000 105000
120000 105000
135000 105000
0 120000
15000 120000
30000 120000
45000 120000
60000 120000
75000 120000
9000

In [13]:
import pandas as pd
df = pd.concat(my_dfs)

In [14]:
gdf=gd.GeoDataFrame(df, geometry=df['geometry'], crs = gdfin.crs)

In [16]:
gdf.head()

,H,V,sY,sX,geometry
0,16,4,0,0,"POLYGON ((-150584.000 2564804.000, -150584.000..."
0,16,4,1,0,"POLYGON ((-150584.000 2579804.000, -150584.000..."
0,16,4,2,0,"POLYGON ((-150584.000 2594804.000, -150584.000..."
0,16,4,3,0,"POLYGON ((-150584.000 2609804.000, -150584.000..."
0,16,4,4,0,"POLYGON ((-150584.000 2624804.000, -150584.000..."


In [15]:
import pickle

# Pickle the DataFrame
with open('my_dataframe.pkl', 'wb') as f:
    pickle.dump(gdf, f)

# # Unpickle the DataFrame
# with open('my_dataframe.pkl', 'rb') as f:
#     df = pickle.load(f)

# # Display the DataFrame
# print(df)
